In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance

from imblearn.over_sampling import RandomOverSampler 

from functions.Pre_processing_Casos_Graves import *
from functions.Pre_processing_models import *

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection._split import StratifiedKFold

from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

## Criação das funções para viabilizar a aplicação do modelo:

#### Função para instanciar o modelo:

In [5]:
from sklearn.neural_network import MLPClassifier

def instantiate_MLP_model (X_train, y_train):

    param_grid = {
        "hidden_layer_sizes": [(1000, 500, 250), (1000, 500)],
        "activation": ["logistic", "tanh", "identity", "relu"],
        "solver": ["lbfgs", "sgd", "adam"],
        #"solver": ["sgd", "adam"],
        "max_iter": [500]
    }
    
    # Init classifier
    mlp_cl = MLPClassifier()

    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits = 5, n_repeats = 10, random_state = 1)

    # Init Grid Search
    grid_cv = GridSearchCV(mlp_cl, param_grid, n_jobs = -1, cv=5, scoring = "roc_auc")

    # Fit 
    grid_cv.fit(X_train, y_train)

    mean_test_score = grid_cv.cv_results_['mean_test_score'].mean()
    std_test_score = grid_cv.cv_results_['std_test_score'].mean()
    best_score = grid_cv.best_score_
    best_params = grid_cv.best_params_

    #print(f"mean_test_score: {grid_cv.cv_results_['mean_test_score']}")
    #print(f"std_test_score: {grid_cv.cv_results_['std_test_score']}")
    #print('')

    print('#'*40)
    #print(f"mean_test_score: {grid_cv.cv_results_['mean_test_score'].mean()}")
    #print(f"std_test_score: {grid_cv.cv_results_['std_test_score'].mean()}")
    print('')
    print(f'best_score: {grid_cv.best_score_}')
    print(f'best_params_: {grid_cv.best_params_}')
    print('#'*40)
    print('')

    return mean_test_score, std_test_score, best_score, best_params

#### Criação de uma função para predição do classificador final:

In [4]:
from sklearn.neural_network import MLPClassifier

def create_final_cl(X_over, y_over, X_test, y_test, hidden_layer_sizes, activation, solver, max_iter):

    result_accuracy = []
    result_f1 = []
    result_precision = []
    result_recall = []
    result_roc_auc = []

    # parametrizando o modelo final
    final_cl = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter)

    #treinando o modelo final
    final_cl.fit(X_over, y_over)

    preds = final_cl.predict(X_test)

    accuracy_score(y_test, preds)

    print('"""Accuracy"""')
    print('accuracy_score:', accuracy_score(y_test, preds))
    print('#'*40)
    print('"""Métricas f1_score:"""')
    print('f1: ', f1_score(y_test, preds))
    print('macro: ', f1_score(y_test, preds, average='macro'))
    print('micro: ', f1_score(y_test, preds, average='micro'))
    print('weighted: ', f1_score(y_test, preds, average='weighted'))
    print('#'*40)
    print('"""Métricas precision_score:"""')
    print('Precision: ', precision_score(y_test, preds))
    print('macro: ', precision_score(y_test, preds, average='macro'))
    print('micro: ', precision_score(y_test, preds, average='micro'))
    print('weighted: ', precision_score(y_test, preds, average='weighted'))
    print('#'*40)
    print('"""Métricas recall_score:"""')
    print('Recall: ', recall_score(y_test, preds))
    print('macro: ', recall_score(y_test, preds, average='macro'))
    print('micro: ', recall_score(y_test, preds, average='micro'))
    print('weighted: ', recall_score(y_test, preds, average='weighted'))
    print('#'*40)
    print('"""ROC_AUC Score:"""')
    print('roc_auc_score: ', roc_auc_score(y_test, preds, multi_class='ovr'))

    result_accuracy.append(accuracy_score(y_test, preds))

    result_f1.append(f1_score(y_test, preds))
    result_f1.append(f1_score(y_test, preds, average='macro'))
    result_f1.append(f1_score(y_test, preds, average='micro'))
    result_f1.append(f1_score(y_test, preds, average='micro'))

    result_precision.append(precision_score(y_test, preds))
    result_precision.append(precision_score(y_test, preds, average='macro'))
    result_precision.append(precision_score(y_test, preds, average='micro'))
    result_precision.append(precision_score(y_test, preds, average='weighted'))

    result_recall.append(precision_score(y_test, preds))
    result_recall.append(precision_score(y_test, preds, average='macro'))
    result_recall.append(precision_score(y_test, preds, average='micro'))
    result_recall.append(precision_score(y_test, preds, average='weighted'))

    result_roc_auc.append(roc_auc_score(y_test, preds, multi_class='ovr'))

    return preds, final_cl, result_accuracy, result_f1, result_precision, result_recall, result_roc_auc

#### Criação de funções para plotagem de métricas:

In [4]:
def plot_conf_matrix (X_test, y_test, preds, final_cl):

    confusion_matrix(y_test, preds)
    matrix = plot_confusion_matrix(final_cl, X_test, y_test, cmap=plt.cm.Blues)
    plt.show()
    print(classification_report(y_test, preds))


def plot_roc_curve(y_test, preds):
    
    print('roc_auc_score: ', roc_auc_score(y_test, preds, multi_class='ovr'))

    fpr, tpr, _ = metrics.roc_curve(y_test,  preds)
    auc = metrics.roc_auc_score(y_test, preds)
    plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
    plt.legend(loc=4)
    plt.show()

def plot_permutation_importance(final_cl, X_test, y_test):
    perm_importance = permutation_importance(final_cl, X_test, y_test)

    sorted_idx = perm_importance.importances_mean.argsort()
    plt.figure(figsize=(10,10))
    plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx], height=0.8)
    plt.xlabel("Permutation Importance")
    plt.show()

#### Criação de uma função para aplicar o modelo para o período de tempo escolhido:

In [5]:
def apply_periodos (d, df):
    
    results_accuracy = []
    results_f1 = []
    results_precision = []
    results_recall = []
    results_roc_auc = []

    if d == 90:

        for i in range (0,758,30):
    
            #treino!
            data_inicio = df['data_notificacao'][0] + pd.Timedelta(days = i)
            data_fim_treino = data_inicio + pd.Timedelta(days = int(d/2))
            data_fim = data_inicio + pd.Timedelta(days = d)
        
            treino = df[(df['data_notificacao'] >= data_inicio) & (df['data_notificacao'] <= data_fim_treino )]
            X_train = treino.drop(labels=['death', 'data_notificacao'], axis=1)
            y_train = treino['death']

            #teste!
            teste = df[(df['data_notificacao'] > data_fim_treino) & (df['data_notificacao'] <= data_fim )]
            X_test = teste.drop(labels=['death', 'data_notificacao'], axis=1)
            y_test = teste['death']

            oversample = RandomOverSampler(sampling_strategy='minority')
            X_over, y_over = oversample.fit_resample(X_train, y_train)

            print('Início do período: ', data_inicio)
            print('Final do período: ', data_fim)

            ####################################################
            #              APLICANDO O MODELO                  #
            ####################################################

            list_random_states = [0, 12, 42, 50, 54, 68, 79, 81, 92, 112]

            mean_test_scores = []
            std_test_scores = []
            best_scores = []
            best_params = []

            for random_state in list_random_states:

                mean_test_score, std_test_score, best_score, best_param = instantiate_MLP_model(X_over, y_over)

                mean_test_scores.append(mean_test_score)
                std_test_scores.append(std_test_score)
                best_scores.append(best_score)
                best_params.append(best_param)

            idx = best_scores.index(max(best_scores))
            best_param = best_params[idx]
        
            preds, final_cl, result_accuracy, result_f1, result_precision, result_recall, result_roc_auc = create_final_cl(X_over, y_over, X_test, y_test, best_param['hidden_layer_sizes'], best_param['activation'], best_param['solver'], best_param['max_iter'])
        
            results_accuracy.append(result_accuracy)
            results_f1.append(result_f1)
            results_precision.append(result_precision)
            results_recall.append(result_recall)
            results_roc_auc.append(result_roc_auc)

            plot_conf_matrix (X_test, y_test, preds, final_cl)

            plot_roc_curve(y_test, preds)

            plot_permutation_importance(final_cl, X_test, y_test)

        return results_accuracy, results_f1, results_precision, results_recall, results_roc_auc

    if d == 120:

        for i in range(0,728,30):

            #treino!
            data_inicio = df['data_notificacao'][0] + pd.Timedelta(days = i)
            data_fim_treino = data_inicio + pd.Timedelta(days = int(d/2))
            data_fim = data_inicio + pd.Timedelta(days = d)

            treino = df[(df['data_notificacao'] >= data_inicio) & (df['data_notificacao'] <= data_fim_treino )]
            X_train = treino.drop(labels=['death', 'data_notificacao'], axis=1)
            y_train = treino['death']

            #teste!
            teste = df[(df['data_notificacao'] >= data_fim_treino) & (df['data_notificacao'] <= data_fim )]
            X_test = teste.drop(labels=['death', 'data_notificacao'], axis=1)
            y_test = teste['death']

            oversample = RandomOverSampler(sampling_strategy='minority')
            X_over, y_over = oversample.fit_resample(X_train, y_train)

            print('Início do período: ', data_inicio)
            print('Final do período: ', data_fim)

            ####################################################
            #              APLICANDO O MODELO                  #
            ####################################################

            list_random_states = [0, 12, 42, 50, 54, 68, 79, 81, 92, 112]

            mean_test_scores = []
            std_test_scores = []
            best_scores = []
            best_params = []

            for random_state in list_random_states:

                mean_test_score, std_test_score, best_score, best_param = instantiate_MLP_model(X_over, y_over)

                mean_test_scores.append(mean_test_score)
                std_test_scores.append(std_test_score)
                best_scores.append(best_score)
                best_params.append(best_param)
                
            idx = best_scores.index(max(best_scores))
            best_param = best_params[idx]
        
            preds, final_cl, result_accuracy, result_f1, result_precision, result_recall, result_roc_auc = create_final_cl(X_over, y_over, X_test, y_test, best_param['hidden_layer_sizes'], best_param['activation'], best_param['solver'], best_param['max_iter'])
        
            results_accuracy.append(result_accuracy)
            results_f1.append(result_f1)
            results_precision.append(result_precision)
            results_recall.append(result_recall)
            results_roc_auc.append(result_roc_auc)

            plot_conf_matrix (X_test, y_test, preds, final_cl)

            plot_roc_curve(y_test, preds)

            plot_permutation_importance(final_cl, X_test, y_test)
        
        return results_accuracy, results_f1, results_precision, results_recall, results_roc_auc

## Lendo o dataset:

In [6]:
df = pd.read_csv('..\..\Data\Casos Graves\df_casos_graves')

### Utilizando a class de pré-processamento para preparar o dataset:

In [7]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino','sexo_Masculino', 'data_notificacao']
features = np.array(list_features)

In [8]:
Pre_processing_models_df = Pre_processing_models(df)
Pre_processing_models_df.run_graves()

df = Pre_processing_models_df.df
df = df[list_features]
df['data_notificacao'] = pd.to_datetime(df['data_notificacao'])
df = df.sort_values(by='data_notificacao', ascending=True)
df = df.reset_index()
df = df.drop("index", axis=1)
df

C:\Users\maril\AppData\Local\Temp\ipykernel_32896\3432879523.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_notificacao'] = pd.to_datetime(df['data_notificacao'])


,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino,data_notificacao
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0.356522,0,1,2020-02-26
1,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0.278261,0,1,2020-02-26
2,1,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0.217391,1,0,2020-02-27
3,1,0,1,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0.278261,0,1,2020-02-27
4,1,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0.260870,1,0,2020-02-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,1,0,2022-06-12
36431,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.556522,0,1,2022-06-12
36432,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.573913,1,0,2022-06-12
36433,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0,1,2022-06-13


### Aplicação do modelo considerando períodos de 90 dias, deslocando cada período em 30 dias:

In [9]:
results_accuracy_90_mlp, results_f1_90_mlp, results_precision_90_mlp, results_recall_90_mlp, results_roc_auc_90_mlp = apply_periodos(90, df)

Início do período:  2020-02-26 00:00:00
Final do período:  2020-05-26 00:00:00


c:\Users\maril\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


########################################

best_score: 0.9373185697659254
best_params_: {'activation': 'tanh', 'hidden_layer_sizes': (1000, 500, 250), 'solver': 'lbfgs'}
########################################



KeyboardInterrupt: 

### Lista com as métricas de cada período:

In [ ]:
list_periodos_90 = [['26/02/2020 até 26/05/2020'], ['27/03/2020 até 25/06/2020'], ['26/04/2020 até 25/07/2020'], ['26/05/2020 até 24/08/2020'], 
['25/06/2020 até 23/09/2020'], ['25/07/2020 até 23/10/2020'], ['24/08/2020 até 22/11/2020'], ['23/09/2020 até 22/12/2020'], ['23/10/2020 até 21/01/2021'], 
['22/11/2020 até 20/02/2021'], ['22/12/2020 até 22/03/2022'], ['21/01/2021 até 21/04/2021'], ['20/02/2021 até 21/05/2021'], ['22/03/2021 até 20/06/2021'], 
['21/04/2021 até 20/07/2021'], ['21/05/2021 até 19/08/2021'], ['20/06/2021 até 18/09/2021'], ['20/07/2021 até 18/10/2021'], ['19/08/2021 até 17/11/2021'], 
['18/09/2021 até 17/12/2021'], ['18/10/2021 até 16/01/2022'], ['17/11/2021 até 15/02/2022'], ['17/12/2021 até 17/03/2022'], ['16/01/2022 até 16/04/2022'], 
['15/02/2022 até 16/05/2022'], ['17/03/2022 até 15/06/2022']]

In [ ]:
metricas_accuracy_90_gb = pd.DataFrame(results_accuracy_90_gb)
metricas_accuracy_90_gb.columns = ['accuracy score']
#metricas_accuracy_90_gb
metricas_f1_90_gb = pd.DataFrame(results_f1_90_gb)
metricas_f1_90_gb.columns = ['f1 score', 'f1 score macro', 'f1 score micro', 'f1 score weighted']
#metricas_f1_90_gb
metricas_precision_90_gb = pd.DataFrame(results_precision_90_gb)
metricas_precision_90_gb.columns = ['precision score', 'precision score macro', 'precision score micro', 'precision score weighted']
#metricas_precision_90_gb
metricas_recall_90_gb = pd.DataFrame(results_recall_90_gb)
metricas_recall_90_gb.columns = ['recall score', 'recall score macro', 'recall score micro', 'recall score weighted']
#metricas_recall_90_gb
metricas_roc_auc_90_gb = pd.DataFrame(results_roc_auc_90_gb)
metricas_roc_auc_90_gb.columns = ['roc_auc score']
#metricas_roc_auc_90_gb
metricas_90_gb = pd.concat([metricas_accuracy_90_gb, metricas_f1_90_gb, metricas_precision_90_gb, metricas_recall_90_gb, metricas_roc_auc_90_gb], axis=1)
metricas_90_gb

In [ ]:
coluna_periodos90 = pd.DataFrame(list_periodos_90)
coluna_periodos90.columns = ['Períodos']
#coluna_periodos90
metricas_90_gb_final = pd.concat([coluna_periodos90, metricas_90_gb], axis=1)
metricas_90_gb_final

In [ ]:
metricas_90_gb_final.sort_values(by=['roc_auc score'], ascending=False)

In [ ]:
metricas_90_gb_final.sort_values(by=['f1 score macro', 'roc_auc score'], ascending=False)

### Aplicação do modelo considerando períodos de 120 dias, deslocando cada período em 30 dias:

In [ ]:
results_accuracy_120_gb, results_f1_120_gb, results_precision_120_gb, results_recall_120_gb, results_roc_auc_120_gb = apply_periodos(120, df)

### Lista com as métricas de cada período:

In [ ]:
list_periodos_120 = [['26/02/2020 até 25/06/2020'], ['27/03/2020 até 25/07/2020'], ['26/04/2020 até 24/08/2020'], ['26/05/2020 até 23/09/2020'], 
['25/06/2020 até 23/10/2020'], ['25/07/2020 até 22/11/2020'], ['24/08/2020 até 22/12/2020'], ['23/09/2020 até 21/01/2021'], ['23/10/2020 até 20/02/2021'], 
['22/11/2020 até 22/03/2021'], ['22/12/2020 até 21/04/2021'], ['21/01/2021 até 21/05/2021'], ['20/02/2021 até 20/06/2021'], ['22/03/2021 até 20/07/2021'], 
['21/04/2021 até 19/08/2021'], ['21/05/2021 até 18/09/2021'], ['20/06/2021 até 18/10/2021'], ['20/07/2021 até 17/11/2021'], ['19/08/2021 até 17/12/2021'], 
['18/09/2021 até 16/01/2022'], ['18/10/2021 até 15/02/2022'], ['17/11/2021 até 17/03/2022'], ['17/12/2021 até 16/04/2022'], ['16/01/2022 até 16/05/2022'], 
['15/02/2022 até 15/06/2022']]

In [ ]:
metricas_accuracy_120_gb = pd.DataFrame(results_accuracy_120_gb)
metricas_accuracy_120_gb.columns = ['accuracy score']
#metricas_accuracy_120_gb
metricas_f1_120_gb = pd.DataFrame(results_f1_120_gb)
metricas_f1_120_gb.columns = ['f1 score', 'f1 score macro', 'f1 score micro', 'f1 score weighted']
#metricas_f1_120_gb
metricas_precision_120_gb = pd.DataFrame(results_precision_120_gb)
metricas_precision_120_gb.columns = ['precision score', 'precision score macro', 'precision score micro', 'precision score weighted']
#metricas_precision_120_gb
metricas_recall_120_gb = pd.DataFrame(results_recall_120_gb)
metricas_recall_120_gb.columns = ['recall score', 'recall score macro', 'recall score micro', 'recall score weighted']
#metricas_recall_120_gb
metricas_roc_auc_120_gb = pd.DataFrame(results_roc_auc_120_gb)
metricas_roc_auc_120_gb.columns = ['roc_auc score']
#metricas_roc_auc_120_gb
metricas_120_gb = pd.concat([metricas_accuracy_120_gb, metricas_f1_120_gb, metricas_precision_120_gb, metricas_recall_120_gb, metricas_roc_auc_120_gb], axis=1)
metricas_120_gb

In [ ]:
coluna_periodos120 = pd.DataFrame(list_periodos_120)
coluna_periodos120.columns = ['Períodos']
#coluna_periodos120.columns
metricas_120_gb_final = pd.concat([coluna_periodos120, metricas_120_gb], axis=1)
metricas_120_gb_final

In [ ]:
metricas_120_gb_final.sort_values(by=['roc_auc score'], ascending=False)

In [ ]:
metricas_120_gb_final.sort_values(by=['f1 score macro', 'roc_auc score'], ascending=False)